In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
# Imports
import pandas as pd
import numpy as np
from scipy.stats import bootstrap
import sys
import warnings
from copy import deepcopy,copy
#from ipywidgets import IntProgress
#from itertools import chain
from IPython.display import display
from datetime import datetime
import pickle
import os
#from sympy import sympify,latex,Float
import random
from math import ceil,sqrt
#import seaborn as sbrn
from scipy.optimize import curve_fit
# Catch stout
#from io import StringIO 
import sys
import pynumdiff
import matplotlib.pyplot as plt
import traceback
# Import Machine Scientist ODE
from importlib.machinery import SourceFileLoader

# Get the absolute path of the script's directory
script_dir = os.path.dirname(os.path.abspath("__file__"))
# Define the relative path to the module
relative_module_path = "rguimera-machine-scientist/machinescientist_ode.py"
path = os.path.join(script_dir, relative_module_path)
ms_ode = SourceFileLoader("ms", path).load_module()

# Import Machine Scientist FIT
from importlib.machinery import SourceFileLoader

# Get the absolute path of the script's directory
script_dir = os.path.dirname(os.path.abspath("__file__"))
# Define the relative path to the module
relative_module_path = "rguimera-machine-scientist/machinescientist_fit.py"
path = os.path.join(script_dir, relative_module_path)
ms_fit = SourceFileLoader("ms_fit", path).load_module()

2025-03-04 13:49:36,121 [INFO] 
Limited Total Variation Regularization Support Detected! 
---> CVXPY is not installed. 
---> Many Total Variation Methods require CVXPY including: 
---> velocity, acceleration, jerk, jerk_sliding, smooth_acceleration
---> Please install CVXPY to use these methods.
---> Recommended to also install MOSEK and obtain a MOSEK license.
You can still use: total_variation_regularization.iterative_velocity

2025-03-04 13:49:36,122 [INFO] 
Limited Linear Model Support Detected! 
---> PYCHEBFUN is not installed. 
---> Install pychebfun to use chebfun derivatives (https://github.com/pychebfun/pychebfun/) 
You can still use other methods 

2025-03-04 13:49:36,123 [INFO] 
Limited Linear Model Support Detected! 
---> CVXPY is not installed. 
---> Install CVXPY to use lineardiff derivatives 
You can still use other methods 



In [ ]:
# Loop over data files

files=[f for f in os.listdir('noise_data/')]
#print(files)
sigmas=[0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.3,0.4,0.5]

def Euler_integral(model,x0,t):
    
    def ode(y):
        # X Lambda input
        predict = model.predict({'A0':pd.DataFrame(data={'B':[y]})})
        return predict['A0'].to_numpy()[0]
    
    x=x0
    t_x=[x0]
    h=(t[1]-t[0])
    for i in range(len(t)-1):
        diff=ode(x)
        x=x+h*diff
        t_x.append(x)
    return np.array(t_x,dtype=np.float64)

def func(x,x0):
    eval=Euler_integral(model_to_fit,x0,t)
    return eval


s_array=[]
ode_lernability=[]
ode_lernability_err=[]
fit_lernability=[]
fit_lernability_err=[]
smooth_lernability=[]
smooth_lernability_err=[]

true_best_error_ode_final=[]
data_best_error_ode_final=[]
data_best_error_ode_final_err=[]

true_best_error_fit_final=[]
data_best_error_fit_final=[]
data_best_error_fit_final_err=[]

true_best_error_smth_final=[]
data_best_error_smth_final=[]
data_best_error_smth_final_err=[]

corpus_true=['(B + ((-(B)) * B))','(B + (-(pow2(B))))','(B + -((B ** 2)))',
             '(((_a1_ + B) * B) / _a1_)','(B + (-(B) ** 2))','((_a1_ * (B ** 2)) + B)',
             '((B + _a5_) * -(B))','-(((_a6_ + B) * B))','(B + ((B ** 2) * _a0_))'
            ]

for sigma in sigmas:
    count_ode=0
    l_ode=0
    count_fit=0
    l_fit=0
    count_smth=0
    l_smth=0
    
    true_best_error_ode=[]
    data_best_error_ode=[]
    
    true_best_error_fit=[]
    data_best_error_fit=[]
    
    true_best_error_smth=[]
    data_best_error_smth=[]
    
    print(sigma)
    
    for file in [f for f in files if f.startswith(str(sigma)+'_')]:
        #Lernability for ODE
        print(file)
        try:
            # True Model ODEint
            data=pd.read_pickle('noise_data/'+file)
            x={}
            y={}
            y['A0']=pd.Series(deepcopy(data['B']))
            x['A0']=deepcopy(data)
            XLABS = ['B']
            params = 8
            """str_model= '(B - (pow2(B))'
            true_model2 = ms_ode.from_string_model(x,y,str_model,1,8,['B'],silence=True)
            str_model= '(B + ((-(B)) * B))'
            true_model1 = ms_ode.from_string_model(x,y,str_model,1,8,['B'],silence=True)
            str_model= '(B + (-(pow2(B))))'
            true_model = ms_ode.from_string_model(x,y,str_model,1,8,['B'],silence=True)"""
            list_dl_trues=[ms_ode.from_string_model(x,y,str_model,1,8,['B'],silence=True) for str_model in corpus_true]
            true_model = min(list_dl_trues, key=lambda x: x.E)
            
            #print(true_model2.E,true_model2.EP,true_model1.E,true_model1.EP,true_model.E,true_model.EP)
            #Load sampled model
            with open(f'./noise_data_res_ODE/{file}', 'rb') as f:
                # A new file will be created
                best_model=pickle.load(f)
            try:
                with open(f'./noise_data_res_exhaustive_new/BMS_{file[:-4]}.pkl', 'rb') as f:
                    # A new file will be created
                    bms_exh=pickle.load(f)
                if best_model.E>bms_exh.E:
                    print('Updating mdl ode model with exhaustive model:',best_model,bms_exh)
                    del best_model
                    best_model = deepcopy(bms_exh)
                    del bms_exh
            except:
                pass
            #best_model.get_energy(bic=True,reset=True)
            print('true',true_model,true_model.canonical(),true_model.E,true_model.variables)
            #print(true_model.par_values)
            #print(true_model.x0[str(true_model)])
            print('bms',best_model,best_model.E)
            #print(best_model.par_values)
            #print(best_model.x0[str(best_model)])
            
            t=data.t.to_numpy()
            model_to_fit=true_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            #int_true=Euler_integral(true_model,true_model.x0[str(true_model)]['A0'],data.t.to_numpy())
            int_true=Euler_integral(true_model,popt[0],data.t.to_numpy())
            
            model_to_fit=best_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            
            #int_best=Euler_integral(best_model,best_model.x0[str(best_model)]['A0'],data.t.to_numpy())
            int_best=Euler_integral(best_model,popt[0],data.t.to_numpy())
            
            
            true_best_err=np.sqrt(np.mean((int_true-int_best)**2))
            data_best_err=np.sqrt(np.mean((data.B.to_numpy()-int_best)**2))
            
            if not np.isinf(int_true[-1]) and not np.isinf(int_best[-1]):

                print(true_best_err,data_best_err)

                true_best_error_ode.append(true_best_err)
                data_best_error_ode.append(data_best_err)
            else:
                print(true_best_err,data_best_err)
                print(popt)
                print(int_true)
                print(int_best)
                print('RMSE error in ODE:')
            if true_best_err>10. or data_best_err>10.:
                print('Check BMS model ODE:',file,str(best_model),best_model.E,true_best_err,data_best_err)
            
            count_ode+=1
            if best_model.E>=true_model.E:
                l_ode+=1
            elif np.isclose(np.float64(best_model.E),np.float64(true_model.E), rtol=1e-05, atol=1e-08):
                l_ode+=1
            else:
                print('non lernable')
                print('ODE***********************',str(true_model),true_model.E,str(best_model),best_model.E)
                print(best_model.par_values)
            #print('ODE',file,'true',true_model.E,'mdl',best_model.E)
        except Exception as e:
            print('Error in ODE:',e)
            print(traceback.format_exc())
            pass
            
        try:
            # True Model Fit deriv
            data=pd.read_pickle('noise_data/'+file)
            x={}
            y={}
            with open(f'./noise_data_res_fit/{file}', 'rb') as f:
                # A new file will be created
                best_model=pickle.load(f)
            try:
                with open(f'./noise_data_res_exhaustive_new/BMS_fit_{file[:-4]}.pkl', 'rb') as f:
                    # A new file will be created
                    bms_exh=pickle.load(f)
                if best_model.E>bms_exh.E:
                    print('Updating mdl ode model with exhaustive model:',best_model,bms_exh)
                    del best_model
                    best_model = deepcopy(bms_exh)
                    del bms_exh
            except:
                pass
            x=best_model.x
            y=best_model.y
            XLABS = ['B']
            params = 8
            """str_model= '(B + ((-(B)) * B))'
            str_model= '(B + (-(pow2(B))))'
            true_model = ms_fit.from_string_model(x,y,str_model,1,8,['B'],silence=True)"""
            
            list_dl_trues=[ms_fit.from_string_model(x,y,str_model,1,8,['B'],silence=True) for str_model in corpus_true]
            true_model = min(list_dl_trues, key=lambda x: x.E)
            #Load sampled model
            t=data.t.to_numpy()
            model_to_fit=true_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            
            int_true=Euler_integral(true_model,popt[0],data.t.to_numpy())
            
            model_to_fit=best_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            
            int_best=Euler_integral(best_model,popt[0],data.t.to_numpy())
            
            
            true_best_err=np.sqrt(np.mean((int_true-int_best)**2))
            data_best_err=np.sqrt(np.mean((data.B.to_numpy()-int_best)**2))

            print('test',int_true[-1])
            print(int_best[-1])
            print(np.isinf(int_true[-1]))
            print(np.isinf(int_best[-1]))
            
            if not np.isinf(int_true[-1]) and not np.isinf(int_best[-1]):
                
                true_best_error_fit.append(true_best_err)
                data_best_error_fit.append(data_best_err)
            else:
                print(true_best_err,data_best_err)
                print(popt)
                #print(int_true)
                #print(int_best)
                print('RMSE error in FIT:')
            if true_best_err>10. or data_best_err>10.:
                print('Check BMS model FIT:',file,str(best_model),best_model.E,true_best_err,data_best_err)
                
            count_fit+=1
            if best_model.E>=true_model.E:
                l_fit+=1
            elif np.isclose(np.float64(best_model.E),np.float64(true_model.E), rtol=1e-05, atol=1e-08):
                l_fit+=1
            else:
                print('FIT***************',str(true_model),true_model.E,str(best_model),best_model.E)
                print(best_model.par_values)
            #print('FIT',file,'true',true_model.E,'mdl',best_model.E)
        except Exception as e:
            print('Error in FIT:',e)
            print(traceback.format_exc())
            print(true_best_err,type(true_best_err))
            print(data_best_err,type(data_best_err))
            pass
            
            
        try:
            # True Model Fit deriv smooth
            data=pd.read_pickle('noise_data/'+file)
            x={}
            y={}
            with open(f'./noise_data_res_fit_smooth/{file}', 'rb') as f:
                # A new file will be created
                best_model=pickle.load(f)
            try:
                with open(f'./noise_data_res_exhaustive_new/BMS_smooth_{file[:-4]}.pkl', 'rb') as f:
                    # A new file will be created
                    bms_exh=pickle.load(f)
                if best_model.E>bms_exh.E:
                    print('Updating mdl ode model with exhaustive model:',best_model,bms_exh)
                    del best_model
                    best_model = deepcopy(bms_exh)
                    del bms_exh
            except:
                pass
            x=best_model.x
            y=best_model.y
            XLABS = ['B']
            params = 8
            
            list_dl_trues=[ms_fit.from_string_model(x,y,str_model,1,8,['B'],silence=True) for str_model in corpus_true]
            true_model = min(list_dl_trues, key=lambda x: x.E)
            #Load sampled model
            
            t=data.t.to_numpy()
            model_to_fit=true_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            
            int_true=Euler_integral(true_model,popt[0],data.t.to_numpy())
            
            model_to_fit=best_model
            popt,pcov=curve_fit(func, [0], data['B'].to_numpy(), p0=[0.01])
            
            int_best=Euler_integral(best_model,popt[0],data.t.to_numpy())
            
            
            true_best_err=np.sqrt(np.mean((int_true-int_best)**2))
            data_best_err=np.sqrt(np.mean((data.B.to_numpy()-int_best)**2))
            
            if not np.isinf(int_true[-1]) and not np.isinf(int_best[-1]):
                
                true_best_error_smth.append(true_best_err)
                data_best_error_smth.append(data_best_err)
            else:
                print(true_best_err,data_best_err)
                print(popt)
                #print(int_true)
                #print(int_best)
                print('RMSE error in SMTH:')
            if true_best_err>10. or data_best_err>10.:
                print('Check BMS model SMTH:',file,str(best_model),best_model.E,true_best_err,data_best_err)
            #Load sampled model
            
            count_smth+=1
            if best_model.E>=true_model.E:
                l_smth+=1
            elif np.isclose(np.float64(best_model.E),np.float64(true_model.E), rtol=1e-05, atol=1e-08):
                l_smth+=1
            else:
                print('SMTH*******************',str(true_model),true_model.E,str(best_model),best_model.E)
                print(best_model.par_values)
            #print('SMTH',file,'true',true_model.E,'mdl',best_model.E)
        except Exception as e:
            print('Error in SMOOTH:',e)
            print(traceback.format_exc())
            print(true_best_err,type(true_best_err))
            print(data_best_err,type(data_best_err))
            
            pass
            
    # appending lernability fracction
    s_array.append(sigma)
    ode_lernability.append(np.divide(float(l_ode),float(count_ode)))
    print(len([1]*l_ode+[0]*(count_ode-l_ode)),[1]*l_ode+[0]*(count_ode-l_ode))
    ode_lernability_err.append(bootstrap(([1]*l_ode+[0]*(count_ode-l_ode),), np.mean, confidence_level=0.95, method='percentile').standard_error)
    fit_lernability.append(np.divide(float(l_fit),float(count_fit)))
    fit_lernability_err.append(bootstrap(([1]*l_fit+[0]*(count_fit-l_fit),), np.mean, confidence_level=0.95, method='percentile').standard_error)
    smooth_lernability.append(np.divide(float(l_smth),float(count_smth)))
    smooth_lernability_err.append(bootstrap(([1]*l_smth+[0]*(count_smth-l_smth),), np.mean, confidence_level=0.95, method='percentile').standard_error)
    
    true_best_error_ode_final.append(np.mean(true_best_error_ode))
    data_best_error_ode_final.append(np.mean(data_best_error_ode))
    data_best_error_ode_final_err.append(np.std(data_best_error_ode,ddof=1)/np.sqrt(np.size(data_best_error_ode)))

    true_best_error_fit_final.append(np.mean(true_best_error_fit))
    data_best_error_fit_final.append(np.mean(data_best_error_fit))
    data_best_error_fit_final_err.append(np.std(data_best_error_fit,ddof=1)/np.sqrt(np.size(data_best_error_fit)))

    true_best_error_smth_final.append(np.mean(true_best_error_smth))
    data_best_error_smth_final.append(np.mean(data_best_error_smth))
    data_best_error_smth_final_err.append(np.std(data_best_error_smth,ddof=1)/np.sqrt(np.size(data_best_error_smth)))

0.005
0.005_1.pkl
true (B + -((B ** 2))) -B**2+B -453.924784793266 ['B']
bms (_a7_ * ((_a7_ + B) * B)) -449.812144432682
8.833857011673066e-05 0.0054038358363513155
lernable
test 0.99782656622261
0.99782656622261
False
False
0.005_11.pkl
true (B + -((B ** 2))) -B**2+B -445.635212260140 ['B']
bms (_a6_ * (B * (B + _a6_))) -441.502568677809
1.6091356578449281e-06 0.006024868197727184
lernable
test 0.9978203648042275
0.9978605209569461
False
False
0.005_14.pkl
Updating mdl ode model with exhaustive model: (_a7_ * (((B + _a7_) * _a1_) * B)) ((_a0_ * B) + (_a1_ * (B ** 2)))
true (B + -((B ** 2))) -B**2+B -455.173461489428 ['B']
bms ((_a0_ * B) + (_a1_ * (B ** 2))) -445.062326753928
0.00016567611640382 0.005433895969015983
lernable
test 0.9978051277843496
1.0018914226384283
False
False
0.005_23.pkl
true (B + -((B ** 2))) -B**2+B -460.706678708674 ['B']
bms (B + -((B ** 2))) -460.706678708674
0.0 0.005354787457673466
lernable
test 0.9978270562669705
0.998854317223623
False
False


In [ ]:
data_store={
    's_array':s_array,
    'ode_lernability':ode_lernability,
    'ode_lernability_err':ode_lernability_err,
    'fit_lernability':fit_lernability,
    'fit_lernability_err':fit_lernability_err,
    'smooth_lernability':smooth_lernability,
    'smooth_lernability_err':smooth_lernability_err,
    'true_best_error_ode_final':true_best_error_ode_final,
    'true_best_error_fit_final':true_best_error_fit_final,
    'true_best_error_smth_final':true_best_error_smth_final,
    'data_best_error_ode_final':data_best_error_ode_final,
    'data_best_error_fit_final':data_best_error_fit_final,
    'data_best_error_smth_final':data_best_error_smth_final,
    'data_best_error_ode_final_err':data_best_error_ode_final_err,
    'data_best_error_fit_final_err':data_best_error_fit_final_err,
    'data_best_error_smth_final_err':data_best_error_smth_final_err,
}
with open('Plots/lernability.pkl','wb') as file:
    pickle.dump(data_store,file=file)